**Import bibliotek:**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, f1_score, roc_auc_score

**Import bazy danych:**

In [ ]:
diabetes = pd.read_csv('diabetes.csv')

**Podział danych na zmienne objaśniające (X) i objaśniane (y):**

In [ ]:
X = diabetes.drop(['Diabetic'], axis=1).copy()
y = diabetes['Diabetic'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

**Towrzymy grupę danych numerycznych - num_features:**

In [ ]:
num_features = ['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age']

**Tworzenie pipeline zawierającego różne 'preprocessory' danych:**

In [ ]:
num_preparation = Pipeline(steps=[
                                ('fill_missing', SimpleImputer(strategy='median')),
                                ('polynomial_features', PolynomialFeatures(degree=3)),
                                ('scaler_1', StandardScaler()),
                                ('pca', PCA(n_components=0.95)),
                                ('scaler_2', StandardScaler())
                                ])

**Tworzenie transformatora danych w kolumnach - data_preparation:**

In [ ]:
data_preparation = ColumnTransformer(transformers=[('numeric_preprocessing', num_preparation, num_features)])

**Tworzymy pipeline zawierający - transormator danych w kolumnach i model ML - LogisticRegression:**

In [ ]:
model_pipeline_v1 = Pipeline(steps=[
                                ('preprocessor',data_preparation),
                                ('model', LogisticRegression(max_iter=10000))
                                ])

**Trenujemy model - model_pipeline_v1:**

In [ ]:
model_pipeline_v1.fit(X_train,y_train)

**Tworzymy funkcję calculate_metrics:**

In [ ]:
metrics_dataframe = pd.DataFrame(columns = ['Model', 'F1_score', 'AUC'])
metrics_dataframe
models = []
models_names = []
predictions_proba_list = []

def calculate_metrics(model, name, X_checked, y_checked):
    models.append(model)
    models_names.append(name)
    global metrics_dataframe
    predictions = model.predict(X_checked)
    predictions_proba = model.predict_proba(X_checked)
    predictions_proba_list.append(predictions_proba[:,1])

    f1_metric = f1_score(y_checked, predictions)
    auc_metric = roc_auc_score(y_checked, predictions_proba[:,1])
    new_row = pd.DataFrame([{'Model': name, 'F1_score': f1_metric, 'AUC': auc_metric}])
    metrics_dataframe = pd.concat([metrics_dataframe, new_row], ignore_index=True)
    return metrics_dataframe

**Uruchamiamy funkcję calculate_metrics dla modelu: model_pipeline_v1:**

In [ ]:
calculate_metrics(model_pipeline_v1, 'Logistic Regression - pipeline', X_test, y_test)

**Tworzymy nowy podział danych treningowych i testowych:**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[['Pregnancies', 'Age']], y, test_size=0.30, random_state=0, stratify=y)

scaler_2var = StandardScaler()
X_train_standardized = scaler_2var.fit_transform(X_train)
X_test_standardized = scaler_2var.transform(X_test)

**Tworzymy funkcję generate_model_LR:**

In [ ]:
model_list=[]

def generate_model_LR(penalty,C):
    global model_list
    
    if penalty=="l1":
        model = LogisticRegression(penalty=penalty,C=C,solver='liblinear')
    elif penalty=="l2":
        model = LogisticRegression(penalty=penalty,C=C)
    elif penalty=="elasticnet":
        model = LogisticRegression(penalty=penalty,C=C,solver='saga', l1_ratio=0.1)
    else:
        raise ValueError("Nieprawidłowa wartość 'penalty'")
    
    name = f"LogisticRegression_penalty:{penalty},C:{C}"
    model_list.append((model, name))
    return model, name

**Tworzymy zbiory parametrów: penalties oraz parametr C oraz generujemy modele funkcją generate_model_LR:**

In [ ]:
penalties = ['l1','l2','elasticnet']
parametrs_C = [0.01,0.1,1,10,100]

for penalty in penalties:
    for C in parametrs_C:
      generate_model_LR(penalty,C)

**Trenujemy w/w modele:**

In [ ]:
for model, name in model_list:
    model.fit(X_train_standardized,y_train)

**Uruchamiamy funkcję calculate_metrics dla nowo wygenerownych modeli:**

In [ ]:
for model, name in model_list:
    metrics_dataframe = calculate_metrics(model, name, X_test_standardized, y_test)

display(metrics_dataframe)

**Wnioski:**

Wygenerowany pipeline uzyskał lepszy parametr F1, ponadto automatyzuje czynności takie jak uzupełnianie braków danych przez wprowadzenie wartości równej medianie dla zbioru treningowego dla wartości numerycznych. Ponadto pipeline zawiera w odróżnieniu do później wygenerowanych modeli funkcję wielomianu stopnia 3 oraz funkcję PCA, być może dlatego uzyskuje lepszy rezultat dla danych testowych. Być może pipeline ma lepsze wyniki różnież ze względu na różnice w parametrze max_iter - dla pipeline jest to 10000 a dla pozostałych modeli to domyślna wartość: 100.